# Lab 12: Resting state analyses

The goal of this lab is to introduce how to do simple resting state analyses. The scripts we provide allow for doing simple seed-based analyses that generate a statistical map of functional connectivity with a seed, or to compute an ROI-ROI correlation matrix from a set of ROIs. 

For this lab some commands may be better run in the terminal rather than through the notebook. Look out for instructions on this throughout the notebook.


### Set up the computing environment

In [ ]:
%%bash

# ICA-AROMA
mkdir ${HOME}/packages
cd ${HOME}/packages
wget https://github.com/maartenmennes/ICA-AROMA/archive/v0.4.4-beta.tar.gz
tar -xf v0.4.4-beta.tar.gz
chmod +x ./ICA-AROMA-0.4.4-beta/*.py
echo 'export PATH=${HOME}/packages/ICA-AROMA-0.4.4-beta:${PATH}' >> ~/.bashrc
source ~/.bashrc

In [ ]:
%%bash

# create a python2.7 environment
conda create -n RestingState python=2.7
source activate RestingState

In [ ]:
%%bash 

# Install necessary python packages

source activate RestingState
conda install -y future numpy pandas seaborn jupyter


### Get octave installed (public form of matlab)

* Run the following in the terminal

In [ ]:
%%bash

# Setup debian environment for octave

echo "deb http://ftp.debian.org/debian jessie-backports main" | sudo tee -a /etc/apt/sources.list
sudo apt update
sudo apt -t jessie-backports install liboctave-dev octave

In [ ]:
%%bash

# Setup Octave

# Open Octave 
octave
# type the following in the gui
pkg install -forge image
# close octave (can type exit at the command prompt)

In [ ]:
%%bash

# make an rc file for octave so that it loads the package image everytime it's opened
touch ~/.octaverc
echo "pkg load image" > ~/.octaverc

### Download example data
* Note it is in bids format, which the scripts will expect

In [ ]:
%%bash

wget --quiet -O 12-Lab_data.tar.gz https://osf.io/2smk3/download &&\
tar -xf 12-Lab_data.tar.gz &&\
rm 12-Lab_data.tar.gz

## Running the scripts 

### Make an ROI
* Let's make visual cortex ROIs from the Van Dijk review that we read
    * L MNI: -30, -88, 0
    * R MNI: 30, -88, 0 

In [ ]:
%%bash

# show usage
/home/brain/RestingState/makeROI.sh


In [ ]:
%%bash

# make an ROIs directory
mkdir ROIs

# run script
/home/brain/RestingState/makeROI.sh -30 -88 0 mm 14 sphere LVis
/home/brain/RestingState/makeROI.sh 30 -88 0 mm 14 sphere RVis

In [ ]:
%%bash 

# take a look
fslview /usr/share/fsl/5.0/data/standard/avg152T1.nii.gz ROIs/LVis.nii.gz ROIs/RVis.nii.gz

In [ ]:
%%bash

# make a list with full paths to these ROIs
find "$PWD/ROIs" -name "*.nii.gz" -type f > ${PWD}/ROIs/roilist.txt

### Run a seed-based analyses with these ROIs

In [ ]:
%%bash

/home/brain/RestingState/processRestingState_bids_wrapper.sh \
--epi=/home/brain/Labs/12-Lab/sub-999/ses-activepre/func/sub-999_ses-activepre_task-rest_rec-pu_run-01_bold.nii.gz \
--roiList=${PWD}/ROIs/roilist.txt \
--compcor > output.log

### Output

In [ ]:
%%bash

# get seedmaps

# run in the terminal

# James can you help modify seedVoxelCorrelation to map to the derivatives directory paths to find rsparams 
# above step that is needed to find files and help with mapping seedmap to standard space
/home/brain/RestingState/seedVoxelCorrelation.sh --epi=/home/brain/Labs/12-Lab/sub-999/ses-activepre/func/sub-999_ses-activepre_task-rest_rec-pu_run-01_bold.nii.gz \
 --roiList=${PWD}/ROIs/roilist.txt \
 --motionscrub --seedmaps --clobber

In [ ]:
# get roi-roi correlation matrix 

# use getroicorr (which is run by run_getroicorr, this script needs some tlc to set up paths)
# output should be a group correlation matrix and a sub x roi-pair list

# Problem set

1) Make two new seeds from the Van Dijk paper using the makeROI script, and run them on the data using the seedVoxelCorrelation script. Show a figure of one of the seedmaps.

In [ ]:
# your code here

2) Use your code from lab 02 to make a timeseries plot of the resting time course from the two visual cortex ROIs, and another plot of two ROIs that should not show high resting state correlations. 

In [ ]:
# your code here